In [1]:
import requests
import pymongo
import pandas as pd

In [68]:
client = pymongo.MongoClient("mongodb://13.209.70.163:27017")
client

MongoClient(host=['13.209.70.163:27017'], document_class=dict, tz_aware=False, connect=True)

In [70]:
# 데이터 베이스 선택
db = client.dss
db

Database(MongoClient(host=['13.209.70.163:27017'], document_class=dict, tz_aware=False, connect=True), 'dss')

In [7]:
db.collection_names()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['user', 'info']

In [8]:
collection = db.info
collection

Collection(Database(MongoClient(host=['13.209.70.163:27017'], document_class=dict, tz_aware=False, connect=True), 'dss'), 'info')

##### find_one : 한 개의 도큐먼트를 가져온다.

- 개별 도큐먼트는 key, value 형태를 띄고 있다

In [9]:
document = collection.find_one( {"subject": "java"} )
document

{'_id': ObjectId('5b59573844a610fd564f0cff'),
 'subject': 'java',
 'level': 4.0,
 'comments': [{'name': 'po', 'msg': 'check'}]}

##### find : 전체 도큐먼트를 가져온다.

- find는 cursor 형태로 리턴한다.
- list(cursor)를 선언하는 순간, cursor 객체 안에 있는 데이터는 사라진다.

In [10]:
documents = collection.find( {"subject": "java"} )
documents

In [11]:
datas = list(documents)
datas

[{'_id': ObjectId('5b59573844a610fd564f0cff'),
  'subject': 'java',
  'level': 4.0,
  'comments': [{'name': 'po', 'msg': 'check'}]},
 {'_id': ObjectId('5b59573844a610fd564f0d03'),
  'subject': 'java',
  'level': 4.0,
  'comments': [{'name': 'alice', 'msg': 'hello'}]}]

In [23]:
list(documents)

[]

##### count : 도큐먼트의 갯수를 가져온다.

In [24]:
documents = collection.find()
documents

In [25]:
documents.count()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


8

##### sort : 도큐먼트를 특정 수순으로 정렬한다

In [26]:
documents = collection.find({ "level": {"$lte":3} }).sort( "level", pymongo.DESCENDING )
list(documents)

[{'_id': ObjectId('5b59573844a610fd564f0d00'),
  'subject': 'javascript',
  'comments': [{'name': 'jin', 'msg': 'nice'}],
  'level': 3.0},
 {'_id': ObjectId('5b59573844a610fd564f0d01'),
  'subject': 'html',
  'comments': [{'name': 'sujin', 'msg': 'I LOVE YOU'},
   {'name': 'sy', 'msg': 'yeah!'}],
  'level': 3.0},
 {'_id': ObjectId('5b59573844a610fd564f0cfe'),
  'subject': 'r',
  'level': 1.0,
  'comments': [{'name': 'jin', 'msg': 'bad'}, {'name': 'alice', 'msg': 'hi'}]},
 {'_id': ObjectId('5b59573844a610fd564f0d02'),
  'subject': 'sass',
  'level': 1.0,
  'comments': [{'name': 'alice', 'msg': 'hello'}]},
 {'_id': ObjectId('5b595a1d83c64912cdfa59ed'),
  'subject': 'less',
  'level': 1.0,
  'comments': [{'name': 'alice', 'msg': 'hello'}]}]

In [28]:
pymongo.DESCENDING, pymongo.ASCENDING

(-1, 1)

##### insert : 데이터를 저장한다

In [30]:
# 단수 데이터 저장

data = {"subject": "css",
        "level": 1,
        "comments": [{"name":"peter", "msg":"hello"}]}
result = collection.insert_one(data)
result

In [31]:
print(result.inserted_id)

5b5967d6faed3626d1776fa0


In [33]:
# 복수 데이터 저장

data = [data = {"subject": "python", "level": 2, "comments": [{"name":"Kevin", "msg":"halo"}]},
        data = {"subject": "javascript", "level": 5, "comments": [{"name":"Jio", "msg":"pretty"}]},
        data = {"subject": "mongodb", "level": 3, "comments": [{"name":"Jane", "msg":"good"}]}]
result = collection.insert_many(data)
result

SyntaxError: invalid syntax (<ipython-input-33-76371a18ae2e>, line 3)

##### 직방 데이터 크롤링 후 저장

In [71]:
zigbang = client.crawling.zigbang
zigbang

Collection(Database(MongoClient(host=['13.209.70.163:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'zigbang')

##### 강사님

In [72]:
url = "https://api.zigbang.com/v3/items?detail=true&item_ids=[11845934,11759101,12146554,12247700,12170322,12181511,12185946,12195176,12016750,12129123,12236544,12075243,12207509,12206185,12257458,12133011,12252976,12226634,12268716,12150608,12103695,12195196,12240584,12236559,12217207,12264847,12257484,12068922,12075266,12182904,12262511,12080274,12225719,12252795,12197909,12107681,12117328,12247394,11998679,12154891,12112034,12255857,11620927,12116471,12236526,12182871,12254303,12036696,11766014,12225326,12206956,12088339,12241830,12169127,12206845,12209390,12250269,12199973,11852255,12156741]"
response = requests.get(url)
response

<Response [200]>

In [73]:
zigbang_dict_list = response.json().get("items")
len(zigbang_dict_list)

58

In [74]:
zigbang_dict_list[0]

{'title': '서울시 마포구 성산동',
 'header': False,
 'header_height': 0,
 'item': {'id': 11845934,
  'images': [{'index': 0,
    'count': 1,
    'url': 'https://ic.zigbang.com/ic/users/4164282/photos/uploads/d28be44d262101448d9a60a889ff74cc767caddc.280602.PNG'},
   {'index': 1,
    'count': 2,
    'url': 'https://ic.zigbang.com/ic/users/4164282/photos/uploads/cde25b63604ef88f76b4409eb11d65c370181fdb.252246.PNG'},
   {'index': 2,
    'count': 3,
    'url': 'https://ic.zigbang.com/ic/users/4164282/photos/uploads/3017cc59ef4c3ddbd4dc482dff6d025129237b18.252360.PNG'},
   {'index': 3,
    'count': 4,
    'url': 'https://ic.zigbang.com/ic/users/4164282/photos/uploads/ca2f2d6ff58f03955d36b1e9d3003cabc6c5da34.254731.PNG'},
   {'index': 4,
    'count': 5,
    'url': 'https://ic.zigbang.com/ic/users/4164282/photos/uploads/155b22a5e2c1a7209b6bcac02dc014f98f9604f5.199810.jpg'}],
  'is_realestate': True,
  'is_direct': False,
  'is_room': False,
  'is_type_room': False,
  'rent': 0,
  'deposit': 13000,
  'i

In [75]:
items = [item["item"] for item in zigbang_dict_list ]
len(items)

58

In [76]:
zigbang = client.crawling.zigbang
zigbang

Collection(Database(MongoClient(host=['13.209.70.163:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'zigbang')

In [77]:
zigbang.insert_many(items)

In [60]:
# 렌트 비용이 50만원 이하의 데이터를 추출
QUERY = {"rent":{"$lte":50}}
documents = zigbang.find(QUERY, {"_id":False, "deposit":True, "rent":True, "options":True, "size":True})
documents

In [61]:
# 결과를 판다스의 데이터 프레임으로 만들기
df = pd.DataFrame(list(documents)) 
df.tail()

,deposit,options,rent,size
93,25000,"가스레인지,신발장,싱크대",0,14.0
94,1000,"가스레인지,싱크대",50,12.0
95,3000,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대",25,5.0
96,20000,-,0,13.5
97,23500,"에어컨,가스레인지,신발장,싱크대",0,13.0


##### delete - database

In [64]:
client.drop_database("crawling")

##### delete - collection : 데이터 베이스를 전부 다 지우고 싶을 때

In [65]:
client.crawling.drop_collection("zigbang")

{'ok': 0.0, 'errmsg': 'ns not found', 'code': 26}

In [34]:
zigbang = client.crawling.zigbang
zigbang

Collection(Database(MongoClient(host=['13.209.70.163:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'zigbang')

In [35]:
url = "https://api.zigbang.com/v3/items?detail=true&item_ids=[12147577,12211570,12118560,12256740,12153257,12067136,12244677,12274134,12066042,12238156,12274901,12222783,12056669,12258898,12104456,12105291,11881108,12200175,12256781,12139924,12236302,12268568,12248795,12133748,12165438,12252355,12257195,12245930,12258035,12146032,12262708,12237816,12087003,12018148,12198407,12262911,12266850,12189784,12265635,12225107,12238327,12258954,12213527,12248719,12163199,12192279,12258942,12075353,12238001,12233122,12205650,12245412,12080445,12173316,12258292,12251354,12269049,12229549,12254456,11982095]"

In [36]:
response = requests.get(url)
response

<Response [200]>

##### Parsing - [{}, {}, {}, ...]

In [38]:
zigbang_dict_list = response.json().get("items")
len(zigbang_dict_list)

60

In [41]:
items = [item["item"] for item in zigbang_dict_list]
len(items)

60

In [42]:
zigbang.insert_many(items)

In [43]:
# 렌트비용이 50 이하인 데이터 추출

QUERY = { "rent": {"$lte":50} }
documents = zigbang.find(QUERY)
documents

In [44]:
datas = list(documents)
len(datas)

55

In [47]:
db.collection_names()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['user', 'info']